In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb
import gc


Tensorflow ver. 2.6.0


In [3]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: single_run. Site: PI. Training: False. Training date: earlier
{'training': False, 'inferring': True, 'site': 'PI', 'training_date': 'earlier', 'mode': 'single_run', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 27, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': False, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': True, 'log_filename': 'log.pkl'}


In [4]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 9; repetition_id = 0

	elif config['site'] == 'MT':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 11; repetition_id = 0

	elif config['site'] == 'MS':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 4
			elif config['training_date'] == 'earlier': 
				exp = 5
				exp = 30

		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 12; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 13; repetition_id = 0
	elif config['site'] == 'PI':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 6
			elif config['training_date'] == 'earlier': exp = 27
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 14; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 35; repetition_id = 0

else:
	exp = config['training_experiment_id']
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

27

In [7]:
logger = Logger()


In [8]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


In [9]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [10]:
manager_class

src.manager.multioutput.ManagerSingleRun

In [11]:
exp

27

In [12]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], repetition_n 10


In [13]:
config['start_repetition_id']

0

In [14]:
if config['inferring'] == True:
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    for idx in range(config['start_repetition_id'], repetition_n): # 10 repetitions
    # for idx in range(0, 4): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)

        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        if config['get_multiple_metrics'] == False:
            predictor_fcn = manager.run_predictor
        else:
            predictor_fcn = manager.run_predictor_multiple_metrics
        result = predictor_fcn()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)

        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        del manager
        gc.collect()
        # idx += 1
    

Beginning run number 0
manager.config {'training': False, 'inferring': True, 'site': 'PI', 'training_date': 'earlier', 'mode': 'single_run', 'training_times': 10, 'uncertainty_method': 'pred_entropy_single', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 27, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': False, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': True, 'log_filename': 'log.pkl', 'inference_times': 1, 'dropout_training': False}
D:/Jorge/datasets/deforestation/PI/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

[0.588685   0.70839795 0.63969597 0.03000562 0.12918853 0.36135155
 0.10801705]
threshold:  0.1264681281202558
threshold 0.1264681281202558


ic| label_current_deforestation_test_classified_incorrect.shape: (2685804,)
    predicted_test_classified_incorrect.shape: (2685804,)
ic| TP_H + FN_H + FP_H + TN_H: 2685804
    len(label_mask_current_deforestation_test): 89527556
ic| TP_H + FN_H + FP_H + TN_H: 2685804
    len(label_mask_current_deforestation_test): 89527556


[0.58866213 0.70839826 0.63970436 0.02999975 0.12918448 0.36130764
 0.10801223]
threshold 0.1264681281202558
(89527556,) (89527556,)


ic| self.m_optimal: {'AA': array([0.02999975]),
                     'UEO': array([0.10801223]),
                     'f1': 57.23,
                     'f1_H': array([0.19032044]),
                     'f1_L': array([0.64300357]),
                     'precision_H': array([0.12918448]),
                     'precision_L': array([0.58866213]),
                     'recall_H': array([0.36130764]),
                     'recall_L': array([0.70839826]),
                     'recall_Ltotal': array([0.63970436])}
ic| self.m_audited_optimal: {'f1': array([0.67470988]),
                             'precision': array([0.62236022]),
                             'recall': array([0.7366751])}


cm_audited [[87631423   585748]
 [  345057   965328]]
[0.62236022 0.7366751 ]
Result idx 9: {'uncertainty_result': {'metrics': {'precision_L': array([0.58866213]), 'recall_L': array([0.70839826]), 'recall_Ltotal': array([0.63970436]), 'AA': array([0.02999975]), 'precision_H': array([0.12918448]), 'recall_H': array([0.36130764]), 'UEO': array([0.10801223]), 'f1_L': array([0.64300357]), 'f1_H': array([0.19032044]), 'f1': 57.23}, 'metrics_audited': {'precision': array([0.62236022]), 'recall': array([0.7366751]), 'f1': array([0.67470988])}, 'exp': 27}}
Grid execution idx: 9


In [15]:
error_count

0

In [16]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.54010115]), 'recall_L': array([0.6654695]), 'recall_Ltotal': array([0.57793953]), 'AA': array([0.03000071]), 'precision_H': array([0.12419713]), 'recall_H': array([0.31461637]), 'UEO': array([0.13498966]), 'f1_L': array([0.59626674]), 'f1_H': array([0.17809138]), 'f1': 51.54}, 'metrics_audited': {'precision': array([0.59044367]), 'recall': array([0.70947068]), 'f1': array([0.6445078])}, 'exp': 27}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.67666562]), 'recall_L': array([0.68728453]), 'recall_Ltotal': array([0.5877494]), 'AA': array([0.02999857]), 'precision_H': array([0.2318683]), 'recall_H': array([0.43078782]), 'UEO': array([0.1111777]), 'f1_L': array([0.68193374]), 'f1_H': array([0.30147171]), 'f1': 60.83}, 'metrics_audited': {'precision': array([0.72287202]), 'recall': array([0.73257316]), 'f1': array([0.72769026])}, 'exp': 27}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.78886503]), 'r

In [17]:
print(len(results))

10


In [18]:
results[7]

{'uncertainty_result': {'metrics': {'precision_L': array([0.61893634]),
   'recall_L': array([0.73566036]),
   'recall_Ltotal': array([0.65096971]),
   'AA': array([0.0299964]),
   'precision_H': array([0.15848866]),
   'recall_H': array([0.45533887]),
   'UEO': array([0.12714525]),
   'f1_L': array([0.67226936]),
   'f1_H': array([0.23513461]),
   'f1': 59.05},
  'metrics_audited': {'precision': array([0.65653137]),
   'recall': array([0.76609165]),
   'f1': array([0.70709273])},
  'exp': 27}}

In [19]:
manager.f1

NameError: name 'manager' is not defined

In [ ]:
print(manager.f1)

54.34
